In [1]:
from random import *
from scipy import signal
from matplotlib import pyplot as plt
import numpy as np
import GPy

l_test_set_path = "./StarLightCurves/StarLightCurves_TEST"
m_test_set_path = "./MALLAT/MALLAT_TEST"

# two trained HGP models
l_model_class_names = [1,3]
m_model_class_names = [3,6]

# perc of Testing data used for HGP sample size
# do not work 
l_sample_size_perc = 0.5
m_sample_size_perc = 0.5

# outlier class names
l_outlier_class_names = [2]
m_outlier_class_names = [7]

# perc of normal and abnormal items
# normal_prec + outlier_prec = 1
# normal_perc = 0.8
outlier_prec = 0.1
# total_test_prec = 0.9

# top_likelihood_rows_rate = 1.0
num_centroids = 3

(Rebbapragada, 2008) mentioned that the shapes of CEPH and RRL are very similar, so they referred to CEPH and RRL as normal classes, and EB was regarded as the anomaly for the known anomalies test. In MALLAT dataset, class 3,6 are normal classes, and class 7,2 are abnormal ones. In this way, they compared the performance of PCAD (their method) with other methods (K-means etc.).

- S1: train two HGP models (model 1 uses CEPH and RRL as training dataset; model 2 uses MALLAT class 3,6 as training dataset)
- S2: build two testing datasets including 95% normal objectives and 5% outliers for OGLE and MALLAT respectively.
- S2: apply model 1 and model 2 to the testing datasets, and measure precision

In [2]:
def selectTopNormalIndex(model_class_names, top_rate):
    top_index = []
    for m_c in model_class_names:
        m_c_index_load = np.loadtxt("sorted_result_model-"+str(m_c)+"class-"+str(m_c)+".csv", delimiter=',', usecols=[0])
        m_c_top_rows_num = int(len(m_c_index_load) * top_rate)
        m_c_top_index_list = m_c_index_load[:m_c_top_rows_num:].tolist()
        top_index.extend(m_c_top_index_list)
    
#     print("top_index length=",len(top_index))
    return top_index
    

In [3]:
def processTestData(test_set_path):
    class_names_test = np.loadtxt(test_set_path, delimiter=',', usecols=[0])
    test_data = np.loadtxt(test_set_path, delimiter=',', usecols=range(1, 1025))
    test_data -= test_data.mean(1)[:,np.newaxis]
    test_data /= test_data.std(1)[:,np.newaxis]
    
    return class_names_test,test_data

In [4]:
# sample_prec doesn't work now
def generateTestData(class_test_names, sample_prec, model_class_names, outlier_class_names, normal_index_range=[],total_test_num=600):
#     print(outlier_class_names)
    if(normal_index_range):
        normal_indices = normal_index_range
    else:
        normal_indices = [i for i,cn in enumerate(class_test_names) if cn in model_class_names]
    abnormal_indices = [i for i,cn in enumerate(class_test_names) if cn in outlier_class_names]
    
#     total_test_indices_num = int(len(normal_indices)*total_test_prec + len(abnormal_indices)*total_test_prec)
#     print("total_test_indices_num=",total_test_indices_num)
    
    
# #     print(abnormal_indices)

#     normal_num = int(normal_perc * total_test_indices_num)
#     abnormal_num = int(outlier_prec * total_test_indices_num)
#     print(normal_num,abnormal_num)
#     print(len(normal_indices))
#     print(len(normal_indices), len(abnormal_indices))

    normal_num = int((1-outlier_prec) * total_test_num)
    abnormal_num = int(outlier_prec * total_test_num)
#     print(normal_num,abnormal_num)
    
    sample_normal_indices = sample(normal_indices, normal_num)
    sample_normal_indices = np.asarray(sample_normal_indices)
    sample_normal_indices = sample_normal_indices.reshape(-1,1)
    sample_normal_indices = sample_normal_indices.astype(int)
    
    
    sample_abnormal_indices = sample(abnormal_indices, abnormal_num)
    sample_abnormal_indices = np.asarray(sample_abnormal_indices)
    sample_abnormal_indices = sample_abnormal_indices.reshape(-1,1)
    sample_abnormal_indices = sample_abnormal_indices.astype(int)
    
#     print(sample_normal_indices, sample_abnormal_indices)
    
    # normal->0; abnormal->1
    normal_indicator = np.zeros(len(sample_normal_indices))
    normal_indicator = normal_indicator.reshape(-1,1)
    
    abnormal_indicator = np.ones(len(sample_abnormal_indices))
    abnormal_indicator = abnormal_indicator.reshape(-1,1)
    
#     print(sample_normal_indices.shape, normal_indicator.shape)
#     print(sample_abnormal_indices.shape, abnormal_indicator.shape)
    normal_array = np.concatenate((sample_normal_indices, normal_indicator), axis=1)
#     print(normal_array)
    abnormal_array = np.concatenate((sample_abnormal_indices, abnormal_indicator), axis=1)
#     print(abnormal_array)
    conbine_array = np.concatenate((normal_array, abnormal_array), axis=0)
#     print(conbine_array)
    
    return conbine_array, normal_num, abnormal_num

In [5]:
def calSumConvergeDis(timeseries_set, cluster, centroids):
    sum_r = 0;
    for i in range(len(timeseries_set)):
        j = int(cluster[i])
#         print(j, timeseries_set[i])
        corr = np.sum(signal.correlate(timeseries_set[i], centroids[j], mode='same', method='fft')/ len(timeseries_set[i]))
        sum_r = sum_r + corr ** 2 * 0.5
#     print(sum_r)
    return sum_r

def calDistance(timeseries_set, centroids):
    maxcorr = np.ones(len(timeseries_set)) * -9999
    best_centroids = np.ones(len(timeseries_set)) * -1
    
    for i in range(len(timeseries_set)):
        best_center = -1
        for j in range(len(centroids)):
            corr = np.sum(signal.correlate(timeseries_set[i], centroids[j], mode='same', method='fft')/ len(timeseries_set[i]))
            if maxcorr[i] < corr:
                maxcorr[i] = corr
                best_center = j
        best_centroids[i] = best_center
    
#     print(best_centroids)
    return best_centroids

def initCentroids(timeseries_set,num_centroids):
    random_indexes = sample(list(range(len(timeseries_set))), num_centroids)
#     print("We initialize the centorids by indexes:", random_indexes)
    
#     rand_centroids = np.random.rand(num_centroids,sample_time_index_num)
#     print("initial_centroids\n",rand_centroids)
    return timeseries_set[random_indexes]

def recalCentroids(timeseries_set, cluster, centroids):
    new_centroids = np.array(centroids)
    for c in range(len(centroids)):
        if len(timeseries_set[cluster==c]) <= 0:
            new_centroids[c] = centroids[c];
        else:
            new_centroids[c] = np.mean(timeseries_set[cluster==c], axis=0)
        
    return new_centroids

In [6]:
def calLikelihood(test_data, indices, model_class_names, sample_size, method):
#     centroids = initCentroids(time_series_set, num_centroids)
#     cluster = np.ones(len(time_series_set)) * -1
#     E_wc = 0
#     converge_iter_up = 20
#     up_iteration = 1000
#     r = 0

#     rand - cc
#     cluster = calDistance(time_series_set,centroids)
#     print(cluster, centroids)
#     while r < converge_iter_up and up_iteration>0:
#         cluster = calDistance(time_series_set,centroids)
#         centroids = recalCentroids(time_series_set, cluster, centroids)

#         E_wc_temp = calSumConvergeDis(time_series_set, cluster, centroids)

#         if E_wc_temp > E_wc:
# #             print(E_wc_temp)
#             r = 0
#             E_wc = E_wc_temp

#         r = r+1
#         up_iteration=up_iteration-1


#     print(centroids, "\n",cluster)

    indices = [int(i) for i in indices]
    time_series_set = test_data[indices]
    
    if(method == 'spcad'):
        file_path_cen = "./StarLight_files/pcad_save_files/model_save" + str(model_class_names) + str(sample_size) + ".npy"
        centroids = np.load(file_path_cen);
        cluster = calDistance(time_series_set,centroids)
    else:
        file_path_cen = "./StarLight_files/pcad_save_files/model_save_r" + str(model_class_names) + str(sample_size) + ".npy"
        centroids = np.load(file_path_cen);
        cluster = calDistance(time_series_set,centroids)
    
    
    score = np.ones(len(indices))
    score = score.reshape(-1,1)

    n = len(time_series_set)
    for index in range(len(time_series_set)):
        for j in range(len(centroids)):
            c_j_len = len(cluster[cluster==j])
            corr = np.sum(signal.correlate(time_series_set[index], centroids[j], mode='same', method='fft')/ len(time_series_set[index]))
            score[index] = score[index] + (c_j_len / n) * corr ** 2


        
    return score

In [7]:
def calLikelihood_mallat(test_data, indices, model_class_names, sample_size, method):

    indices = [int(i) for i in indices]
    time_series_set = test_data[indices]
    
    if(method == 'spcad'):
        file_path_cen = "./MALLAT_files/pcad_save_files/model_save" + str(model_class_names) + str(sample_size) + ".npy"
        centroids = np.load(file_path_cen);
        cluster = calDistance(time_series_set,centroids)
    else:
        file_path_cen = "./MALLAT_files/pcad_save_files/model_save_r" + str(model_class_names) + str(sample_size) + ".npy"
        centroids = np.load(file_path_cen);
        cluster = calDistance(time_series_set,centroids)
    
    
    score = np.ones(len(indices))
    score = score.reshape(-1,1)

    n = len(time_series_set)
    for index in range(len(time_series_set)):
        for j in range(len(centroids)):
            c_j_len = len(cluster[cluster==j])
            corr = np.sum(signal.correlate(time_series_set[index], centroids[j], mode='same', method='fft')/ len(time_series_set[index]))
            score[index] = score[index] + (c_j_len / n) * corr ** 2


        
    return score

sorted_result_array format

| index                               | indicator               | log_predictive_density |
|-------------------------------------|-------------------------|:----------------------:|
| sampled indexes of  testing dataset | normal(0) / abnormal(1) |                        |

In [8]:
def sortLikelihood(test_array, likelihood):
    likelihood = np.asarray(likelihood)
    likelihood = likelihood.reshape(-1,1)

    combine_result = np.concatenate((test_array, likelihood), axis=1)
    sorted_result = np.sort(combine_result.view('f8,f8,f8'), order=['f2'], axis=0).view(np.float)
    
#     print(sorted_result)
    
#     result_file_name = "sorted_result_model.csv";
#     np.savetxt(result_file_name, sorted_result, delimiter=",",fmt='%d,%d,%1.9f')
    return sorted_result

In [9]:
def calPrecision(sorted_result_array, abnormal_num):
    detect_abnormal_num_array = sorted_result_array[0:abnormal_num,:]
#     print(detect_abnormal_num_array.shape)
    detect_abnormal_num = np.sum(detect_abnormal_num_array[:,1])
    return detect_abnormal_num / abnormal_num

test_array format

| index                               | indicator               |
|-------------------------------------|-------------------------|
| sampled indexes of  testing dataset | normal(0) / abnormal(1) |

In [10]:
# S2: build testing datasets
l_class_names_test, l_test_data = processTestData(l_test_set_path)
m_class_names_test, m_test_data = processTestData(m_test_set_path)


In [21]:
top_likelihood_rows_rate = 0.8
total_test_num = 5000
# method = 'rand-c'
method = 'spcad'
for i in range(30):
    for sample_size in [0.1]:   

        # for light-curve data
        l_normal_index_range = selectTopNormalIndex(l_model_class_names, top_likelihood_rows_rate)
        l_normal_index_range = [int(i) for i in l_normal_index_range]
        l_test_array, l_normal_num, l_abnormal_num = generateTestData(l_class_names_test, l_sample_size_perc, l_model_class_names, l_outlier_class_names, l_normal_index_range,total_test_num)

        l_likelihood = calLikelihood(l_test_data, l_test_array[:,0],l_model_class_names,sample_size,method)
        l_sorted_result_array = sortLikelihood(l_test_array, l_likelihood)
        np.savetxt("l_sorted_result_array.csv", l_sorted_result_array, delimiter=",",fmt='%d,%d,%1.9f')
        l_precision = calPrecision(l_sorted_result_array, l_abnormal_num)
        print('sample_size=',sample_size,'method=',method,' total_test_num=',total_test_num,' normal_num=',l_normal_num,' abnormal_num=',l_abnormal_num,' precision=',l_precision)

print('complete')

sample_size= 0.1 method= spcad  total_test_num= 5000  normal_num= 4500  abnormal_num= 500  precision= 0.114
sample_size= 0.1 method= spcad  total_test_num= 5000  normal_num= 4500  abnormal_num= 500  precision= 0.118
sample_size= 0.1 method= spcad  total_test_num= 5000  normal_num= 4500  abnormal_num= 500  precision= 0.134
sample_size= 0.1 method= spcad  total_test_num= 5000  normal_num= 4500  abnormal_num= 500  precision= 0.096
sample_size= 0.1 method= spcad  total_test_num= 5000  normal_num= 4500  abnormal_num= 500  precision= 0.11
sample_size= 0.1 method= spcad  total_test_num= 5000  normal_num= 4500  abnormal_num= 500  precision= 0.102
sample_size= 0.1 method= spcad  total_test_num= 5000  normal_num= 4500  abnormal_num= 500  precision= 0.112
sample_size= 0.1 method= spcad  total_test_num= 5000  normal_num= 4500  abnormal_num= 500  precision= 0.114
sample_size= 0.1 method= spcad  total_test_num= 5000  normal_num= 4500  abnormal_num= 500  precision= 0.12
sample_size= 0.1 method= spcad

In [13]:
total_test_num = 600
# method = 'spcad'
method = 'rand-c'
# for i in range(30):
for sample_size in [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:   

    # for mallat data
    m_test_array, m_normal_num, m_abnormal_num = generateTestData(m_class_names_test, m_sample_size_perc, m_model_class_names, m_outlier_class_names, [], total_test_num)
    m_likelihood = calLikelihood_mallat(m_test_data, m_test_array[:,0], m_model_class_names, sample_size, method)
    m_sorted_result_array = sortLikelihood(m_test_array, m_likelihood)
    np.savetxt("m_sorted_result_array.csv", m_sorted_result_array, delimiter=",",fmt='%d,%d,%1.9f')
    m_precision = calPrecision(m_sorted_result_array, m_abnormal_num)
    print('sample_size=',sample_size,'method=',method,'total_test_num=',total_test_num,' normal_num=',m_normal_num,' abnormal_num=',m_abnormal_num,' precision=',m_precision)
print('complete')

sample_size= 0.3 method= rand-c total_test_num= 600  normal_num= 540  abnormal_num= 60  precision= 0.0166666666667
sample_size= 0.4 method= rand-c total_test_num= 600  normal_num= 540  abnormal_num= 60  precision= 0.0166666666667
sample_size= 0.5 method= rand-c total_test_num= 600  normal_num= 540  abnormal_num= 60  precision= 0.0166666666667
sample_size= 0.6 method= rand-c total_test_num= 600  normal_num= 540  abnormal_num= 60  precision= 0.0166666666667
sample_size= 0.7 method= rand-c total_test_num= 600  normal_num= 540  abnormal_num= 60  precision= 0.0333333333333
sample_size= 0.8 method= rand-c total_test_num= 600  normal_num= 540  abnormal_num= 60  precision= 0.0
sample_size= 0.9 method= rand-c total_test_num= 600  normal_num= 540  abnormal_num= 60  precision= 0.0166666666667
sample_size= 1.0 method= rand-c total_test_num= 600  normal_num= 540  abnormal_num= 60  precision= 0.0
complete
